In [1]:
import matplotlib.pyplot as plt
import imgaug as ia
import os
from imgaug import augmenters as iaa
import numpy as np
from scipy import misc
from glob import glob
import json
%matplotlib inline

In [2]:
# Load in images *ALL*
PROCESS_ALL_IMAGES = True
dataset = "24-06-2017"
bibpics = glob("/Users/Alex/Desktop/extracted_features/%s/**/Bib_%s.jpg" % (dataset, ("*" if PROCESS_ALL_IMAGES else "*11011")))
outdir = "/Users/Alex/Desktop/augmented_features/%s" % dataset
if not os.path.exists(outdir):
    os.makedirs(outdir)
images = [misc.imread(bib) for bib in bibpics]

In [3]:
def affine():
    TRANSLATE_PCT_RANGE = 0.2
    ROTATION_RANGE = (-45, 45)
    SHEAR_RANGE = (-10,10) 
    
    translate_percent = {
        "x": (-TRANSLATE_PCT_RANGE, +TRANSLATE_PCT_RANGE),
        "y": (-TRANSLATE_PCT_RANGE, +TRANSLATE_PCT_RANGE),
    }
    rotate=ROTATION_RANGE
    shear=SHEAR_RANGE
    mode = "edge"
    
    return iaa.Affine(translate_percent=translate_percent,
                      rotate=rotate,
                      shear=shear,
                      mode=mode)

def superpixels():
    return iaa.Superpixels(n_segments=250, p_replace=0.25)

def add_neg():
    return iaa.Add((-45, 0))

def add_pos():
    return iaa.Add((0, 45))

def mul_neg():
    return iaa.Multiply((0.5, 1))

def mul_pos():
    return iaa.Multiply((1, 1.5))

In [4]:
# Sometimes(0.5, ...) applies the given augmenter in 50% of all cases,
# e.g. Sometimes(0.5, GaussianBlur(0.3)) would blur roughly every second image.
def sometimes(aug, pct = 0.5):
    return iaa.Sometimes(pct, aug)
    
def one_of(funcs):
    return iaa.OneOf(funcs)

# Define our sequence of augmentation steps that will be applied to every image
# All augmenters with per_channel=0.5 will sample one value _per image_
# in 50% of all cases. In all other cases they will sample new values
# _per channel_.
seq = iaa.Sequential(
    [
        affine(),
        sometimes(superpixels(), 0.1),
        sometimes(one_of([add_pos(), add_neg()])),
        sometimes(one_of([mul_pos(), mul_neg()])),
    ],
    random_order=True
)

In [5]:
bibpics_aug = seq.augment_images(images)
if PROCESS_ALL_IMAGES:
    for original_file_loc, aug_bib in zip(bibpics, bibpics_aug):
        file_id = os.path.basename(original_file_loc)
        misc.imsave("%s/%s" % (outdir, file_id), aug_bib)
else:
    plt.imshow(bibpics_aug[0])

In [6]:
# # execute 0 to 5 of the following (less important) augmenters per image
#         # don't execute all of them, as that would often be way too strong
#         iaa.SomeOf((0, 5),
#             [
#                 sometimes(iaa.Superpixels(p_replace=(0, 1.0), n_segments=(150, 200))), # convert images into their superpixel representation
#                 iaa.OneOf([
#                     iaa.GaussianBlur((0, 3.0)), # blur images with a sigma between 0 and 3.0
#                     iaa.AverageBlur(k=(2, 7)), # blur image using local means with kernel sizes between 2 and 7
#                     iaa.MedianBlur(k=(3, 11)), # blur image using local medians with kernel sizes between 2 and 7
#                 ]),
#                 iaa.Sharpen(alpha=(0, 1.0), lightness=(0.75, 1.5)), # sharpen images
#                 iaa.Emboss(alpha=(0, 1.0), strength=(0, 2.0)), # emboss images
#                 # search either for all edges or for directed edges
#                 sometimes(iaa.OneOf([
#                     iaa.EdgeDetect(alpha=(0, 0.7)),
#                     iaa.DirectedEdgeDetect(alpha=(0, 0.7), direction=(0.0, 1.0)),
#                 ])),
#                 iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.5), # add gaussian noise to images
#                 iaa.OneOf([
#                     iaa.Dropout((0.01, 0.1), per_channel=0.5), # randomly remove up to 10% of the pixels
#                     iaa.CoarseDropout((0.03, 0.15), size_percent=(0.02, 0.05), per_channel=0.2),
#                 ]),
#                 iaa.Add((-10, 10), per_channel=0.5), # change brightness of images (by -10 to 10 of original value)
#                 iaa.Multiply((0.5, 1.5), per_channel=0.5), # change brightness of images (50-150% of original value)
#                 iaa.ContrastNormalization((0.5, 2.0), per_channel=0.5), # improve or worsen the contrast
#                 iaa.Grayscale(alpha=(0.0, 1.0)),
#                 sometimes(iaa.ElasticTransformation(alpha=(0.5, 3.5), sigma=0.25)), # move pixels locally around (with random strengths)
#                 sometimes(iaa.PiecewiseAffine(scale=(0.01, 0.05))) # sometimes move parts of the image around
#             ],
#             random_order=True
#         )